In [2]:
import pickle

import numpy as np
import matplotlib.pyplot as plt

import torchvision
import torch
import torch.utils.data as Data

In [7]:
# load data
def load_CIFAR_batch(filename):
    with open(filename, 'rb')as f:
        datadict = pickle.load(f,encoding='latin1')
        X = datadict['data']
        Y = datadict['labels']
        X = X.reshape(10000, 3, 32, 32)
        Y = np.array(Y)
        return X, Y

def unpickle_labels(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

#加载标签名字
labels = unpickle_labels("./data/batches.meta")

#加载训练集
train_X=[]
train_Y=[]
for i in range(5):
    s = './data/data_batch_' + str(i+1)
    train_X1, train_Y1 = load_CIFAR_batch("./data/data_batch_1")
    train_X.append(train_X1)
    train_Y.append(train_Y1)
    
train_X = np.concatenate((train_X[0],train_X[1],train_X[2],train_X[3],train_X[4]),axis=0)
train_Y = np.concatenate((train_Y[0],train_Y[1],train_Y[2],train_Y[3],train_Y[4]),axis=0)   

#加载验证集
test_X,test_Y = load_CIFAR_batch('./data/test_batch')


#验证结果
print(train_X.shape)
print(train_Y.shape)
print(test_X.shape)
print(test_Y.shape)

print(labels[b'label_names'])

(50000, 3, 32, 32)
(50000,)
(10000, 3, 32, 32)
(10000,)
[b'airplane', b'automobile', b'bird', b'cat', b'deer', b'dog', b'frog', b'horse', b'ship', b'truck']


In [16]:
# 第一个图像的可视化


# 超参数
BATCH_SIZE = 5

#修饰
train_X = torch.from_numpy(train_X).float()
train_Y = torch.from_numpy(train_Y)
test_X = torch.from_numpy(test_X).float()
test_Y = torch.from_numpy(test_Y)


#加载
train_dataset = Data.TensorDataset(train_X, train_Y)
train_loader = Data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2,) # load data

train_batch = next(iter(train_loader))
images,labels = train_batch

#方法一： 定义一个转换成图片的函数
to_pil_image = torchvision.transforms.ToPILImage()
image = to_pil_image(images[0])
image.show()


#方法二：
#from PIL import Image
#images = images.numpy()
#i0 = Image.fromarray(images[0][0]).convert('L')        # convert L表示转换成luminosity channel
#i1 = Image.fromarray(images[0][1]).convert('L')
#i2 = Image.fromarray(images[0][2]).convert('L')
#img = Image.merge("RGB",(i0,i1,i2))
#img.save('my_pic','png')

#方法三
#grid=torchvision.utils.make_grid(images,nrow=10)
#plt.figure(figsize=(16,10))
#plt.imshow(np.transpose(grid,(1,2,0)))

ValueError: pic should be 2/3 dimensional. Got 1 dimensions.

In [8]:
class KNN(object):
    def __init__(self,train_X,train_y,K,RGB=False):
        self.X=train_X
        self.y=train_y
        self.K=K
        self.RGB = RGB
        
    def greyDistance(self,x1,x2):         #只考虑单色图像，即只考虑形状特征
        return np.linalg.norm(x1[:1024]-x2[:1024],2)

    def distance(self,x1,x2):         #考虑颜色和形状特征
        return np.linalg.norm(x1-x2,2)
    
    def prediction(self, test_X):
        fun = self.distance if self.RGB else self.greyDistance
        dis = np.zeros((test_X.shape[0],self.X.shape[0]))             # m行n列，m表示验证集的数据个数,n表示共有可以作比较的n个点
        for i in range(dis.shape[0]):
            for j in range(dis.shape[1]):
                dis[i][j] = fun(test_X[i],self.X[j])
        pos = np.argsort(dis,axis=1)    #argsort函数返回的是数组值从小到大的索引值，-表示降序排列，axis=1表示按行操作
        pos = pos[:,0:self.K]
        y_pred = self.y[pos]
        y_pred = np.sum(y_pred,axis=1)/self.K
        y_pred = np.round(y_pred)
        return y_pred   
    
    def error(self,y_pred,test_y):
        return sum(y_pred!=test_y)/len(test_y)

In [12]:
err=[]
K=[5,15,25]

#数据预处理,这里我选的数据比较少，因为训练实在太慢，结果不好。。。
train_X = train_X[:20000].reshape(20000,-1)[:,:1024]
train_Y = train_Y[:20000].reshape(20000,-1)[:,:1024]
test_X = test_X[:10000].reshape(10000,-1)[:,:1024]
test_Y = test_Y[:10000].reshape(10000,-1)[:,:1024]

print(train_X.shape)

train_X = pca(train_X,16*16)
train_Y = pca(train_Y,16*16)
test_X = pca(test_X,16*16)
test_Y = pca(test_Y,16*16)

print(train_X.shape)

(20000, 256)


In [15]:
# knn算法
for k in K:
    print('train k = ',k,'...')
    knn = KNN(train_X,train_Y,k,RGB=True)
    pred_Y = knn.prediction(test_X)
    this_err = knn.error(pred_Y,test_Y)
    err.append(this_err)
print('training finished')
print(err)

train k =  5 ...


KeyboardInterrupt: 